In [121]:
using UrlDownload, Plots, HTTP, CSV, DataFrames, Random, BenchmarkTools, BetaML
import DecisionTree, Flux


url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data";
feature_names = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "max_HR", "exang", "oldpeak", "slope", "ca", "thal","heart_dis"]
data2 = urldownload(url, true, format = :CSV, header = feature_names);
h_data = DataFrame(data2)
describe(h_data)


,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,age,54.4389,29.0,56.0,77.0,0,Float64
2,sex,0.679868,0.0,1.0,1.0,0,Float64
3,cp,3.15842,1.0,3.0,4.0,0,Float64
4,trestbps,131.69,94.0,130.0,200.0,0,Float64
5,chol,246.693,126.0,241.0,564.0,0,Float64
6,fbs,0.148515,0.0,0.0,1.0,0,Float64
7,restecg,0.990099,0.0,1.0,2.0,0,Float64
8,thalach,149.607,71.0,153.0,202.0,0,Float64
9,exang,0.326733,0.0,0.0,1.0,0,Float64


In [122]:
replace!(h_data.ca,"?" => "0") #converting missing values to null

303-element Vector{String}:
 "0.0"
 "3.0"
 "2.0"
 "0.0"
 "0.0"
 "0.0"
 "2.0"
 "0.0"
 "1.0"
 "0.0"
 "0.0"
 "0.0"
 "1.0"
 ⋮
 "0.0"
 "0.0"
 "2.0"
 "0.0"
 "0.0"
 "2.0"
 "0.0"
 "0.0"
 "2.0"
 "1.0"
 "1.0"
 "0"

In [123]:
replace!(h_data.thal,"?" => "0")

303-element Vector{String}:
 "6.0"
 "3.0"
 "7.0"
 "3.0"
 "3.0"
 "3.0"
 "3.0"
 "3.0"
 "7.0"
 "7.0"
 "6.0"
 "3.0"
 "6.0"
 ⋮
 "3.0"
 "6.0"
 "7.0"
 "3.0"
 "3.0"
 "6.0"
 "7.0"
 "7.0"
 "7.0"
 "7.0"
 "3.0"
 "3.0"

In [124]:
h_data.ca = parse.(Float64, h_data.ca)
h_data.thal = parse.(Float64, h_data.thal)    #changing them to float so the BetaML modelling work without issues

303-element Vector{Float64}:
 6.0
 3.0
 7.0
 3.0
 3.0
 3.0
 3.0
 3.0
 7.0
 7.0
 6.0
 3.0
 6.0
 ⋮
 3.0
 6.0
 7.0
 3.0
 3.0
 6.0
 7.0
 7.0
 7.0
 7.0
 3.0
 3.0

In [125]:
describe(h_data) # just to check that all the variables has right type and 

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,age,54.4389,29.0,56.0,77.0,0,Float64
2,sex,0.679868,0.0,1.0,1.0,0,Float64
3,cp,3.15842,1.0,3.0,4.0,0,Float64
4,trestbps,131.69,94.0,130.0,200.0,0,Float64
5,chol,246.693,126.0,241.0,564.0,0,Float64
6,fbs,0.148515,0.0,0.0,1.0,0,Float64
7,restecg,0.990099,0.0,1.0,2.0,0,Float64
8,thalach,149.607,71.0,153.0,202.0,0,Float64
9,exang,0.326733,0.0,0.0,1.0,0,Float64


In [126]:
replace!(h_data.heart_dis,1 => 2, 3 => 2, 4 => 2) # now 2 is present

303-element Vector{Int64}:
 0
 2
 2
 0
 0
 0
 2
 0
 2
 2
 0
 0
 2
 ⋮
 0
 2
 2
 2
 0
 2
 2
 2
 2
 2
 2
 0

In [127]:
replace!(h_data.heart_dis,0 => 1) # now 1 is absent

303-element Vector{Int64}:
 1
 2
 2
 1
 1
 1
 2
 1
 2
 2
 1
 1
 2
 ⋮
 1
 2
 2
 2
 1
 2
 2
 2
 2
 2
 2
 1

In [128]:
describe(h_data) #now prediction values are only 1 & 2 (1 is absent and 2 is present)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,age,54.4389,29.0,56.0,77.0,0,Float64
2,sex,0.679868,0.0,1.0,1.0,0,Float64
3,cp,3.15842,1.0,3.0,4.0,0,Float64
4,trestbps,131.69,94.0,130.0,200.0,0,Float64
5,chol,246.693,126.0,241.0,564.0,0,Float64
6,fbs,0.148515,0.0,0.0,1.0,0,Float64
7,restecg,0.990099,0.0,1.0,2.0,0,Float64
8,thalach,149.607,71.0,153.0,202.0,0,Float64
9,exang,0.326733,0.0,0.0,1.0,0,Float64


In [129]:
x = Matrix{Union{Missing,Float64,String}}(h_data[:,1:13]);
y = Vector{Int64}(h_data[:,14]);

In [130]:
xFull = predictMissing(x,rng=copy(FIXEDRNG)).X̂;

Iter. 1:	Var. of the post  14.720620309270092 	  Log-likelihood -8654.178838662394


In [131]:
y_ohe  = oneHotEncoder(y);

In [132]:
((xtrain,xtest),(xtrainFull,xtestFull),(ytrain,ytest),(ytrain_oh,ytest_oh)) = partition([x,xFull,y,y_ohe],[0.8,1-0.8],rng=copy(FIXEDRNG));

In [133]:
myForest = buildForest(xtrain,ytrain,30, rng=copy(FIXEDRNG),forceClassification=true);

In [134]:
ŷtrain,ŷtest   = predict.(Ref(myForest), [xtrain,xtest],rng=copy(FIXEDRNG));

In [135]:
trainAccuracy,testAccuracy  = accuracy.([parse.(Int64,mode(ŷtrain,rng=copy(FIXEDRNG))),parse.(Int64,mode(ŷtest,rng=copy(FIXEDRNG)))],[ytrain,ytest])

2-element Vector{Float64}:
 1.0
 0.8032786885245902

In [136]:
cm = ConfusionMatrix(parse.(Int64,mode(ŷtest,rng=copy(FIXEDRNG))),ytest,classes=[1,2],labels=["Absent","Present"])
print(cm;what="all")

3×3 Matrix{Any}:
 "Labels"     "Absent"    "Present"
 "Absent"   30           6
 "Present"   6          19

3×3 Matrix{Any}:
 "Labels"    "Absent"   "Present"
 "Absent"   0.833333   0.166667
 "Present"  0.24       0.76


-----------------------------------------------------------------

*** CONFUSION MATRIX ***

Scores actual (rows) vs predicted (columns):


Normalised scores actual (rows) vs predicted (columns):


 *** CONFUSION REPORT ***

- Accuracy:               0.8032786885245902
- Misclassification rate: 0.19672131147540983
- Number of classes:      2

  N Class   precision   recall  specificity  f1Score  actualCount  predictedCount
                          TPR       TNR                 support                  

  1 Absent      0.833    0.833        0.760    0.833           36              36
  2 Present     0.760    0.760        0.833    0.760           25              25

- Simple   avg.    0.797    0.797        0.797    0.797
- Weigthed avg.    0.803    0.803        0.790    0.803

-----------------------------------------------------------------


In [138]:
####Now compare with Decision Tree

In [139]:
# We can train the model
model = DecisionTree.build_forest(ytrain, xtrainFull,-1,30,rng=123)
# we generate predictions and measure their error
(ŷtrain,ŷtest) = DecisionTree.apply_forest.([model],[xtrainFull,xtestFull]);
(trainAccuracy,testAccuracy) = accuracy.([ŷtrain,ŷtest],[ytrain,ytest])

2-element Vector{Float64}:
 0.9834710743801653
 0.819672131147541

In [141]:
########## Neural Network ############

In [142]:
xScaleFactors   = getScaleFactors(xtrainFull)
D               = size(xtrainFull,2)
classes         = unique(y)
nCl             = length(classes)

2

In [143]:
ls   = 50
l1   = DenseLayer(D,ls,f=relu,rng=copy(FIXEDRNG))
l2   = DenseLayer(ls,nCl,f=relu,rng=copy(FIXEDRNG))

DenseLayer([-0.21670047185742425 0.11485258740575172 … 0.015708445469304444 -0.143712203658498; -0.09002285443322663 -0.3106534072484348 … 0.24140674578878807 -0.0467179750052083], [0.03062628460300243, 0.09404664993620965], BetaML.Utils.relu, nothing)

In [144]:
l3   = VectorFunctionLayer(nCl,f=softmax) ## Add a (parameterless) layer whose activation function (softMax in this case) is defined to all its nodes at once

VectorFunctionLayer{0}(fill(NaN), 2, 2, BetaML.Utils.softmax, nothing, nothing)

In [145]:
mynn = buildNetwork([l1,l2,l3],squaredCost,name="Multinomial logistic regression Model heart disease") ## Build the NN and use the MSE as error function 

NN(AbstractLayer[DenseLayer([-0.19687530959080626 0.11544496297917989 … -0.05598223094485649 0.002138517600364709; -0.08178698082600586 -0.09013840862941555 … -0.2700364115819889 0.004476485381907591; … ; -0.025915512537422913 -0.13056448075414953 … -0.026913579804940246 0.26042043933950687; 0.20742643302401387 -0.04244391216027166 … -0.07352930707299968 -0.30143554079789964], [0.2475958451621727, 0.08226907268205336, 0.10388889417149283, -0.11131697180073483, -0.1468940776225006, -0.027258225608090658, -0.07019643251425495, -0.2886476021065192, -0.039210506664476674, -0.06713446918227134  …  -0.1439254733419904, -0.1853363218137804, -0.20352858780757255, 0.0015028967970229234, -0.05504984654735395, -0.2176419341768248, -0.1463224541567973, -0.1449790503871383, 0.15786425167021867, -0.2829014303108609], BetaML.Utils.relu, nothing), DenseLayer([-0.21670047185742425 0.11485258740575172 … 0.015708445469304444 -0.143712203658498; -0.09002285443322663 -0.3106534072484348 … 0.241406745788788

In [146]:
res  = train!(mynn,scale(xtrainFull,xScaleFactors),ytrain_oh,epochs=500,batchSize=8,optAlg=ADAM(),rng=copy(FIXEDRNG)) 

***
*** Training Multinomial logistic regression Model heart disease for 500 epochs with algorithm ADAM.
Training.. 	 avg ϵ on (Epoch 1 Batch 30): 	 0.21186544440644547


Training the Neural Network... 15%|████                 |  ETA: 0:00:06

Training.. 	 avg ϵ on (Epoch 50 Batch 30): 	 0.061405696937562405
Training.. 	 avg ϵ on (Epoch 100 Batch 30): 	 0.023383461041245893


Training.. 	 avg ϵ on (Epoch 150 Batch 30): 	 0.003351196947473489

Training the Neural Network... 29%|███████              |  ETA: 0:00:05

...

Training the Neural Network... 87%|███████████████████  |  ETA: 0:00:01


Training.. 	 avg ϵ on (Epoch 450 Batch 30): 	 0.03128408946951958

Training the Neural Network...100%|█████████████████████| Time: 0:00:06



Training.. 	 avg ϵ on (Epoch 500 Batch 30): 	 0.0001222050782705046
Training of 500 epoch completed. Final epoch error: 0.013490710450436351.


(epochs = 500, ϵ_epochs = [0.28206958169328145, 0.22320560995660887, 0.18842786038522769, 0.16186657666583398, 0.14021825736056967, 0.12536590006558362, 0.11507452104296659, 0.10895461274889606, 0.10453240959591863, 0.1014439758672413  …  0.013500375074344328, 0.013492043221863677, 0.013487380137307105, 0.013701565058436838, 0.013799313154214206, 0.01354581952993218, 0.013530861977241314, 0.013503749608654357, 0.013496202800231921, 0.013490710450436351], θ_epochs = Any[])

In [147]:
(ŷtrain,ŷtest)  = predict.(Ref(mynn),[scale(xtrainFull,xScaleFactors),scale(xtestFull,xScaleFactors)])
trainAccuracy   = accuracy(ŷtrain,ytrain,rng=copy(FIXEDRNG))

0.9793388429752066

In [148]:
testAccuracy    = accuracy(ŷtest,ytest,rng=copy(FIXEDRNG))

0.8524590163934426

In [149]:
cm = ConfusionMatrix(ŷtest,ytest,classes=[1,2],labels=["Absent","Present"],rng=copy(FIXEDRNG))
print(cm)

3×3 Matrix{Any}:
 "Labels"     "Absent"    "Present"
 "Absent"   34           2
 "Present"   7          18

3×3 Matrix{Any}:
 "Labels"    "Absent"   "Present"
 "Absent"   0.944444   0.0555556
 "Present"  0.28       0.72


-----------------------------------------------------------------

*** CONFUSION MATRIX ***

Scores actual (rows) vs predicted (columns):


Normalised scores actual (rows) vs predicted (columns):


 *** CONFUSION REPORT ***

- Accuracy:               0.8524590163934426
- Misclassification rate: 0.14754098360655743
- Number of classes:      2

  N Class   precision   recall  specificity  f1Score  actualCount  predictedCount
                          TPR       TNR                 support                  

  1 Absent      0.829    0.944        0.720    0.883           36              41
  2 Present     0.900    0.720        0.944    0.800           25              20

- Simple   avg.    0.865    0.832        0.832    0.842
- Weigthed avg.    0.858    0.852        0.812    0.849

-----------------------------------------------------------------


In [151]:
##################        FLUX         #######################################################

In [152]:
xtrainT, ytrain_ohT = transpose.([scale(xtrainFull,xScaleFactors), ytrain_oh])
xtestT, ytest_ohT   = transpose.([scale(xtestFull,xScaleFactors), ytest_oh])

2-element Vector{LinearAlgebra.Transpose{Float64, Matrix{Float64}}}:
 [0.8101917340992101 0.03522572756953108 … -1.9575440035067864 0.4780634455864905; -1.4922019523732948 0.6701505774131562 … 0.6701505774131562 0.6701505774131562; … ; 1.365429964786998 -0.7527370318697552 … -0.7527370318697552 -0.7527370318697552; -0.8916976312610534 1.111001457046834 … -0.8916976312610534 -0.8916976312610534]
 [0.0 0.0 … 1.0 1.0; 1.0 1.0 … 0.0 0.0]

In [153]:
Random.seed!(123)

l1         = Flux.Dense(D,ls,Flux.relu)
l2         = Flux.Dense(ls,ls,Flux.relu)
l3         = Flux.Dense(ls,nCl,Flux.relu)
Flux_nn    = Flux.Chain(l1,l3)
loss(x, y) = Flux.logitcrossentropy(Flux_nn(x), y)
ps         = Flux.params(Flux_nn)
nndata     = Flux.Data.DataLoader((xtrainT, ytrain_ohT), batchsize=8,shuffle=true)
begin for i in 1:500  Flux.train!(loss, ps, nndata, Flux.ADAM()) end end
ŷtrain     = Flux.onecold(Flux_nn(xtrainT),1:3)
ŷtest      = Flux.onecold(Flux_nn(xtestT),1:3)
trainAccuracy =  accuracy(ŷtrain,ytrain)

1.0

In [154]:
testAccuracy  = accuracy(ŷtest,ytest)

0.7868852459016393

In [157]:
### All the benchmarks for different models' processing times 

In [158]:
@btime buildForest(xtrain,ytrain,30, rng=copy(FIXEDRNG),forceClassification=true);

  801.809 ms (1990076 allocations: 147.66 MiB)


In [159]:
@btime  DecisionTree.build_forest(ytrain, xtrainFull,-1,30,rng=123);

  4.138 ms (12603 allocations: 1.55 MiB)


In [160]:
@btime train!(mynn,scale(xtrainFull),ytrain_oh,epochs=500,batchSize=8,rng=copy(FIXEDRNG),verbosity=NONE);

  6.285 s (44891975 allocations: 5.27 GiB)


In [161]:
@btime begin for i in 1:500 Flux.train!(loss, ps, nndata, Flux.ADAM()) end end;

  1.031 s (4615000 allocations: 811.72 MiB)
